In [32]:
# AIML_Swiggy’s Restaurant Recommendation System using Streamit

import pandas as pd
import numpy as np

In [33]:
df=pd.read_csv(r"C:\Users\z039692\OneDrive - Alliance\Desktop\GUVI\Swiggy Restaurant Recommendation System using Streamlit\swiggy.csv")
df

,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
0,567335,AB FOODS POINT,Abohar,--,Too Few Ratings,₹ 200,"Beverages,Pizzas",22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json
1,531342,Janta Sweet House,Abohar,4.4,50+ ratings,₹ 200,"Sweets,Bakery",12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json
2,158203,theka coffee desi,Abohar,3.8,100+ ratings,₹ 100,Beverages,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json
3,187912,Singh Hut,Abohar,3.7,20+ ratings,₹ 250,"Fast Food,Indian",22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json
4,543530,GRILL MASTERS,Abohar,--,Too Few Ratings,₹ 250,"Italian-American,Fast Food",12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json
...,...,...,...,...,...,...,...,...,...,...,...
148536,553122,The Food Delight,Yavatmal,--,Too Few Ratings,₹ 200,"Fast Food,Snacks",21522053000452,https://www.swiggy.com/restaurants/the-food-de...,"The Food Delight, 94MC+X35, New Singhania Naga...",Menu/553122.json
148537,562647,MAITRI FOODS & BEVERAGES,Yavatmal,--,Too Few Ratings,₹ 300,Pizzas,license,https://www.swiggy.com/restaurants/maitri-food...,"MAITRI FOODS & BEVERAGES, POLIC MITRYA SOCIETY...",Menu/562647.json
148538,559435,Cafe Bella Ciao,Yavatmal,--,Too Few Ratings,₹ 300,"Fast Food,Snacks",21522251000378,https://www.swiggy.com/restaurants/cafe-bella-...,"Cafe Bella Ciao, SHOP NO 2 NEMANI MARKET SBI S...",Menu/559435.json
148539,418989,GRILL ZILLA,Yavatmal,--,Too Few Ratings,₹ 250,Continental,21521251000241,https://www.swiggy.com/restaurants/grill-zilla...,"GRILL ZILLA, SHO NO 2/6, POSTEL GROUND CHOWPAT...",Menu/418989.json


In [38]:
import pandas as pd
import numpy as np

# Load the raw Swiggy data
df=pd.read_csv(r"C:\Users\z039692\OneDrive - Alliance\Desktop\GUVI\Swiggy Restaurant Recommendation System using Streamlit\swiggy.csv")

# Remove any columns with "license" in the column name (case insensitive)
df = df.loc[:, ~df.columns.str.contains('license', case=False)]

# Remove rows where any cell contains "license" (case insensitive)
df = df[~df.apply(lambda row: row.astype(str).str.contains("license", case=False)).any(axis=1)]

# Clean ratings
df['rating'] = df['rating'].replace('--', np.nan)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Clean rating count (e.g., '100+ ratings' → 100)
df['rating_count'] = df['rating_count'].astype(str).str.extract('(\d+)').astype(float)

# Clean cost (remove ₹ and whitespace)
df['cost'] = df['cost'].replace('[₹, ]', '', regex=True).astype(float)

# Drop rows with essential missing values
df.dropna(subset=['rating', 'rating_count', 'cost', 'cuisine', 'city'], inplace=True)

# Save the cleaned data
df.to_csv("cleaned_data.csv", index=False)

print("✅ Cleaned data saved as cleaned_data.csv (license data removed).")

✅ Cleaned data saved as cleaned_data.csv (license data removed).


In [39]:
import pandas as pd

df = pd.read_csv("cleaned_data.csv")

In [40]:
drop_cols = ['restaurant_id', 'license', 'url', 'address', 'menu_path']
df = df.drop(columns=[col for col in drop_cols if col in df.columns])

In [41]:
categorical_cols = ['name', 'city', 'cuisine']
numerical_cols = ['rating', 'rating_count', 'cost']

In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pickle

# Load the cleaned data
df = pd.read_csv("cleaned_data.csv")

# Optional: reset index to align properly
df.reset_index(drop=True, inplace=True)

# Simplify cuisine: only use first cuisine per row
df['primary_cuisine'] = df['cuisine'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else x)

# Categorical columns
categorical_cols = ['city', 'primary_cuisine']
numerical_cols = ['rating', 'rating_count', 'cost']

# One-hot encode categorical columns
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_cat = encoder.fit_transform(df[categorical_cols])

# Create DataFrame from encoded array
encoded_cat_df = pd.DataFrame(encoded_cat, columns=encoder.get_feature_names_out(categorical_cols))

# Join with numerical features
encoded_final_df = pd.concat([df[numerical_cols], encoded_cat_df], axis=1)

# Save encoded data and encoder
encoded_final_df.to_csv("encoded_data.csv", index=False)
with open("encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

print("✅ Encoding complete. 'encoded_data.csv' and 'encoder.pkl' saved.")

✅ Encoding complete. 'encoded_data.csv' and 'encoder.pkl' saved.


In [43]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
cleaned_df = pd.read_csv("cleaned_data.csv")
encoded_df = pd.read_csv("encoded_data.csv")

# Load encoder
with open("encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

# --- Recommendation Function ---
def recommend_restaurants(selected_restaurant_name, top_n=5):
    try:
        # Find the index of the selected restaurant
        selected_index = cleaned_df[cleaned_df['name'] == selected_restaurant_name].index[0]

        # Compute cosine similarity between selected and all others
        similarity_scores = cosine_similarity(
            [encoded_df.iloc[selected_index]],
            encoded_df
        )[0]

        # Get indices of top similar restaurants (excluding self)
        similar_indices = np.argsort(similarity_scores)[::-1][1:top_n+1]

        # Map back to cleaned data
        recommendations = cleaned_df.iloc[similar_indices][[
            'name', 'city', 'rating', 'rating_count', 'cost', 'cuisine', 'address'
        ]]

        return recommendations.reset_index(drop=True)
    
    except IndexError:
        print("❌ Error: Selected restaurant not found.")
        return pd.DataFrame()

# --- Example Usage ---
# Get top 5 restaurants similar to 'Domino's Pizza'
result = recommend_restaurants("Domino's Pizza", top_n=5)

print("Top Recommendations:\n")
print(result)

Top Recommendations:

             name             city  rating  rating_count   cost cuisine  \
0  Domino's Pizza  Fatehgarh-sahib     4.4          20.0  400.0  Pizzas   
1  Domino's Pizza           Hospet     4.4          20.0  400.0  Pizzas   
2  Domino's Pizza          Katihar     4.4          20.0  400.0  Pizzas   
3  Domino's Pizza             Moga     4.4          20.0  400.0  Pizzas   
4  Domino's Pizza         Silvassa     4.4          20.0  400.0  Pizzas   

                                             address  
0  Domino's Pizza, Ground Floor, Fateh Complex,si...  
1  Domino's Pizza, Ground Floor, Virat Complex,Bu...  
2  Domino's Pizza, Shop No A5 - Ground Floor, Kat...  
3  Domino's Pizza, UpperGroundFloor,SunnyTowerNea...  
4  Domino's Pizza, Shop No.29, Ground Floor, Pram...  
